В этом ноутбуке обучим модель LAMBERT распознавать спам в email

In [ ]:
model_name = 'lambert'
train_dataset_name = 'spam emails'

In [ ]:
!pip install tensorflow-text
import tensorflow_text as text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 KB 36.6 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully u

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

Загружаем данные

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/data_for_colab/spam_emails.csv', encoding = "ISO-8859-1")
df

Mounted at /content/drive


,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...,...,...
5166,1518,ham,Subject: put the 10 on the ft\r\nthe transport...,0
5167,404,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,2933,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,1409,ham,Subject: industrial worksheets for august 2000...,0


In [ ]:
df.drop(columns=['Unnamed: 0', 'label'], inplace=True)
df.columns = ['DATA_COLUMN', 'IS_SPAM']

In [ ]:
df

,DATA_COLUMN,IS_SPAM
0,Subject: enron methanol ; meter # : 988291\r\n...,0
1,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,"Subject: photoshop , windows , office . cheap ...",1
4,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...
5166,Subject: put the 10 on the ft\r\nthe transport...,0
5167,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,Subject: industrial worksheets for august 2000...,0


In [ ]:
df['DATA_COLUMN'] = df['DATA_COLUMN'].apply(lambda x: x.replace('\r\n', ' ').replace('\n', ' '))

In [ ]:
df

,DATA_COLUMN,IS_SPAM
0,Subject: enron methanol ; meter # : 988291 thi...,0
1,"Subject: hpl nom for january 9 , 2001 ( see at...",0
2,"Subject: neon retreat ho ho ho , we ' re aroun...",0
3,"Subject: photoshop , windows , office . cheap ...",1
4,Subject: re : indian springs this deal is to b...,0
...,...,...
5166,Subject: put the 10 on the ft the transport vo...,0
5167,Subject: 3 / 4 / 2000 and following noms hpl c...,0
5168,Subject: calpine daily gas nomination > > juli...,0
5169,Subject: industrial worksheets for august 2000...,0


In [ ]:
df['IS_SPAM'].value_counts()

0    3672
1    1499
Name: IS_SPAM, dtype: int64

In [ ]:
df_positive = df[df['IS_SPAM']==1]

In [ ]:
df_negative = df[df['IS_SPAM']==0]

Создаем тестовую и обучающую выборки

In [ ]:
# Тестовая выборка
n_test = df_negative.shape[0] // 2
df_negative_test = df_negative.tail(n_test)
n_test = df_positive.shape[0] // 2
df_positive_test = df_positive.tail(n_test)

In [ ]:
df_negative_test.shape

(1836, 2)

In [ ]:
df_positive_test.shape

(749, 2)

In [ ]:
df_positive_test

,DATA_COLUMN,IS_SPAM
2699,Subject: your mother knows you watch girls moa...,1
2702,Subject: 65 % off for all new software . pyrrh...,1
2703,Subject: [ wrenches ] 68 % off dreamweaver mx ...,1
2706,Subject: he reached around and fingered me whi...,1
2709,"Subject: best meds around , paliourg haven ' t...",1
...,...,...
5159,Subject: pictures streamlined denizen ajar cha...,1
5161,Subject: penny stocks are about timing nomad i...,1
5162,Subject: anomaly boys from 3881 uosda apaprove...,1
5164,Subject: slutty milf wants to meet you take th...,1


In [ ]:
df_balanced_test = pd.concat([df_negative_test, df_positive_test])

In [ ]:
df_balanced_test.sample(10)

,DATA_COLUMN,IS_SPAM
4907,"Subject: enron / hpl actuals for june 29 , 200...",0
4910,"Subject: hi apology sorry company bent , summe...",1
3454,"Subject: precis who ' ve mirth paliourg , ! va...",1
3835,Subject: resume - liz bellamy following is the...,0
3892,Subject: beaumont methanol this is to confirm ...,0
2902,"Subject: f . o . m . hpl nom . eff . april 1 ,...",0
3424,Subject: canvas 2005 cable t v de 5 rambler wi...,1
3088,Subject: simple and quick pone line service ca...,1
3637,"Subject: re : 98 - 6719 and 98 - 9676 robert ,...",0
4725,Subject: winning with small - cap daily tradin...,1


In [ ]:
df_balanced_test['IS_SPAM'].value_counts()

0    1836
1     749
Name: IS_SPAM, dtype: int64

In [ ]:
# Обучающая выборка
n_train = df_negative.shape[0] // 2
df_negative_train = df_negative.head(n_train)
n_train = df_positive.shape[0] // 2
df_positive_train = df_positive.head(n_train)

In [ ]:
df_balanced_train = pd.concat([df_negative_train, df_positive_train])

In [ ]:
df_balanced_train['IS_SPAM'].value_counts()

0    1836
1     749
Name: IS_SPAM, dtype: int64

In [ ]:
df_balanced_train.sample(10)

,DATA_COLUMN,IS_SPAM
1491,Subject: re : aquila header ( meter # 980072 )...,0
1330,Subject: txu deal # 108672 could you add meter...,0
2522,Subject: cheap pharmacy st do you want cheap a...,1
1654,Subject: fw : pymt for 03 / 01 sales adjustmen...,0
2374,Subject: digitize your memories with compaq sc...,0
2234,Subject: october flow at meter 982694 zavisch ...,0
2521,Subject: aquila header - 12 / 12 / 00 flow day...,0
2535,Subject: calpine daily gas nomination > ricky ...,0
2435,Subject: let ' s talk about your home loan . d...,0
1669,Subject: same day change - revision # 2 - txu ...,0


In [ ]:
X_train = df_balanced_train['DATA_COLUMN'].squeeze()
y_train = df_balanced_train['IS_SPAM'].squeeze()

In [ ]:
X_test = df_balanced_test['DATA_COLUMN'].squeeze()
y_test = df_balanced_test['IS_SPAM'].squeeze()

Работаем с моделью

In [ ]:
lambert_preprocess = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3') # uses same thing as bert


In [ ]:
lambert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/lambert_en_uncased_L-24_H-1024_A-16/2")

In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = lambert_preprocess(text_input)
outputs = lambert_encoder(preprocessed_text)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [ ]:
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

In [ ]:
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=METRICS)

In [ ]:
history = model.fit(X_train, y_train, epochs=15)

Epoch 1/15
81/81 [==============================] - 96s 898ms/step - loss: 0.3886 - accuracy: 0.8282 - precision: 0.8588 - recall: 0.4873
Epoch 2/15
81/81 [==============================] - 76s 939ms/step - loss: 0.2234 - accuracy: 0.9323 - precision: 0.9184 - recall: 0.8411
Epoch 3/15
81/81 [==============================] - 79s 980ms/step - loss: 0.1761 - accuracy: 0.9485 - precision: 0.9290 - recall: 0.8905
Epoch 4/15
81/81 [==============================] - 82s 1s/step - loss: 0.1507 - accuracy: 0.9524 - precision: 0.9276 - recall: 0.9065
Epoch 5/15
81/81 [==============================] - 82s 1s/step - loss: 0.1362 - accuracy: 0.9547 - precision: 0.9305 - recall: 0.9119
Epoch 6/15
81/81 [==============================] - 83s 1s/step - loss: 0.1264 - accuracy: 0.9574 - precision: 0.9323 - recall: 0.9199
Epoch 7/15
81/81 [==============================] - 84s 1s/step - loss: 0.1173 - accuracy: 0.9613 - precision: 0.9379 - recall: 0.9279
Epoch 8/15
81/81 [============================

In [ ]:
y_predicted = model.predict(X_test)
y_predicted


81/81 [==============================] - 77s 945ms/step


array([[1.4062412e-03],
       [4.4393749e-04],
       [1.6549472e-03],
       ...,
       [9.9907947e-01],
       [5.3579724e-01],
       [5.3235728e-01]], dtype=float32)

In [ ]:
y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [1]])

In [ ]:
y_test

2536    0
2537    0
2539    0
2540    0
2542    0
       ..
5159    1
5161    1
5162    1
5164    1
5170    1
Name: IS_SPAM, Length: 2585, dtype: int64

In [ ]:
accuracy_score(y_test, y_predicted)

0.9500967117988395

In [ ]:
precision_score(y_test, y_predicted)


0.9317548746518106

In [ ]:
recall_score(y_test, y_predicted)

0.8931909212283045

In [ ]:
f1_score(y_test, y_predicted)

0.9120654396728016

In [ ]:
df_results_on_test = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1_score'])

In [ ]:
index = model_name + ' trained on ' + train_dataset_name + ' and tested on ' + train_dataset_name + ' dataset'
df_results_on_test.loc[index, 'accuracy'] = accuracy_score(y_test, y_predicted)
df_results_on_test.loc[index, 'precision'] = precision_score(y_test, y_predicted)
df_results_on_test.loc[index, 'recall'] = recall_score(y_test, y_predicted)
df_results_on_test.loc[index, 'f1_score'] =  f1_score(y_test, y_predicted)

In [ ]:
df_results_on_test

,accuracy,precision,recall,f1_score
lambert trained on spam emails and tested on spam emails dataset,0.950097,0.931755,0.893191,0.912065


Сохраним обученную модель

In [ ]:
saved_model_path = '/content/drive/MyDrive/data_for_colab/lambert_trained_on_spam_emails_19_january'

In [ ]:
print(saved_model_path)

/content/drive/MyDrive/data_for_colab/lambert_trained_on_spam_emails_19_january


In [ ]:
model.save(saved_model_path, include_optimizer=True) 

Сохраним датафреймы с результатами на обучающей выборке и на тестовой выборке

Сначала результаты для обучающей выборки

In [ ]:
name_for_train_csv = model_name + ' trained on ' + train_dataset_name + ' quality on train dataset'

In [ ]:
def calculate_f1_score(precision, recall):
    F1 = 2 * (precision * recall) / (precision + recall)
    return F1

In [ ]:
df_with_train_quality = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1_score'])

In [ ]:
for el in ['accuracy', 'precision', 'recall']:
    df_with_train_quality.loc[name_for_train_csv, el] = history.history[el][-1]
df_with_train_quality.loc[name_for_train_csv, 'f1_score'] = calculate_f1_score(history.history['precision'][-1], history.history['recall'][-1])

In [ ]:
df_with_train_quality

,accuracy,precision,recall,f1_score
lambert trained on spam emails quality on train dataset,0.969052,0.946595,0.946595,0.946595


In [ ]:
df_with_train_quality.to_csv('/content/drive/MyDrive/data_for_colab/dataframes/train_quality/' + name_for_train_csv + '.csv')

Теперь результаты для тестовой выборки

In [ ]:
name_for_test_csv = model_name + ' trained on ' + train_dataset_name + ' and tested on ' + train_dataset_name + ' dataset'

In [ ]:
df_results_on_test.to_csv('/content/drive/MyDrive/data_for_colab/dataframes/test_quality/' + name_for_test_csv + '.csv')